In [ ]:
import pyreadr
import pandas as pd

In [2]:
M_06 = pd.read_csv('../../01_data/02_bibliometrix/06_dataframe_-_improved_affiliations.csv') # also works for RData

/home/ubd/Environments/bibliometry/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6,7,8,9,10,11,17,22,32,39,55,60,66) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Read grid org. names csv file

In [3]:
grid = pd.read_csv('../../01_data/05_org_names/grid.csv') 
institutes = pd.read_csv('../../01_data/05_org_names/full_tables/institutes.csv') 
addresses = pd.read_csv('../../01_data/05_org_names/full_tables/addresses.csv') 
geonames = pd.read_csv('../../01_data/05_org_names/full_tables/geonames.csv') 
aliases = pd.read_csv('../../01_data/05_org_names/full_tables/aliases.csv') 
labels = pd.read_csv('../../01_data/05_org_names/full_tables/labels.csv')
acronyms = pd.read_csv('../../01_data/05_org_names/full_tables/acronyms.csv')
external_ids = pd.read_csv('../../01_data/05_org_names/full_tables/external_ids.csv')
links = pd.read_csv('../../01_data/05_org_names/full_tables/links.csv')
relationships = pd.read_csv('../../01_data/05_org_names/full_tables/relationships.csv')
types = pd.read_csv('../../01_data/05_org_names/full_tables/types.csv')

In [4]:
grid.head()

,ID,Name,City,State,Country
0,grid.1001.0,Australian National University,Canberra,Australian Capital Territory,Australia
1,grid.1002.3,Monash University,Melbourne,Victoria,Australia
2,grid.1003.2,University of Queensland,Brisbane,Queensland,Australia
3,grid.1004.5,Macquarie University,Sydney,New South Wales,Australia
4,grid.1005.4,UNSW Sydney,Sydney,New South Wales,Australia


In [11]:
from collections import Counter
# all_names = df['CompanyName'][df['RegAddress.PostTown']=='CAMBRIDGE'].unique()
all_names = M_06["Organisation"].unique()
names_freq = Counter()
for name in all_names:
    names_freq.update(str(name).split(" "))
key_words = [word for (word,_) in names_freq.most_common(45)]
print(key_words)

['UNIV', '&', 'INST', 'RES', 'CTR', 'TECHNOL', 'SCI', 'NATL', 'AGR', 'ENGN', 'COLL', 'SCH', 'DEV', 'ENERGY', 'ENVIRONM', 'INT', 'LAB', 'LTD', 'TECH', 'STATE', 'RECH', 'POLYTECH', 'FED', 'ECOLE', 'ELECT', 'CO', 'NAT', 'MED', 'MINIST', 'AFRICA', 'ACAD', 'IND', 'DEPT', 'APPL', 'SUPER', 'HIGHER', 'WATER', 'SERV', 'EL', 'SOUTH', 'FAC', 'GRP', 'AFRICAN', 'RESOURCES', 'HLTH']


In [13]:
len(all_names)

16245

In [18]:
M_06.loc[:,["Country", "City", "Organisation"]].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [65]:
unique_country = list()
unique_city = list()
org_pub_freq = list()
org_indexes = list()
for i,x in enumerate(all_names):
    indexes = x == M_06["Organisation"]
    cou = set(M_06.loc[indexes, "Country"])
    city = set(M_06.loc[indexes, "City"])
    pub_freq = len(set(M_06.loc[indexes, "ID"]))
    # if len(cou) > 1 or len(city) > 1:
    #    print("The following are not unified:\n" + cou + "\n" + city )
    unique_country.append(cou)
    unique_city.append(city)
    org_pub_freq.append(pub_freq)
    ## org_indexes.append(indexes)



In [66]:
uni_df = pd.DataFrame({#"aff_indexes":org_indexes,
                      "org_names": all_names,
                      "city": unique_city,
                      "country": unique_country,
                      "pub_freq": org_pub_freq})

0            1
1            1
2            2
3            2
4            3
         ...  
92409    31677
92410    31678
92411    31679
92412    31679
92413    31680
Name: ID, Length: 92414, dtype: int64

In [47]:
[all_names[i] for i,x in enumerate(unique_country) if len(x)>1][1:10]

['FED UNIV TECHNOL AKURE',
 'CABI',
 'FED UNIV TECHNOL',
 'COUNCIL SCI & IND RES CSIR',
 'UNIV SHARJAH',
 'TEXAS A&M UNIV',
 'NATL RES CTR',
 'UPR AIDA',
 'UNIV MONTPELLIER']

In [53]:
len([all_names[i] for i,x in enumerate(unique_country) if len(x)>1])

420

In [34]:
test_bool = M_06["Organisation"] == all_names[1]
M_06.loc[test_bool, "City"].unique()

array(['CAIRO', 'GIZA', 'EL SHIEKH ZAYED', 'EL SHEIKHZAYED',
       'GIZA GOVERNORAT', 'EL SHEIKH ZAYED', 'DOHA', 'ORMAN GIZA',
       'ORMAN', 'GIZA PC'], dtype=object)

# Group names by countries

In [ ]:
all_main_name = pd.DataFrame(columns=['country', 'city', 'names','alias','score'])
all_names.sort()
all_main_name['names'] = all_names
all_main_name['country'] = all_main_name['names'].apply(lambda x: x[0])